# Local Notebook

Desarrollamos en un notebook local para analizar el procedimiento a seguir
para guardar la tabla en un archivo.

In [ ]:
! gcloud auth login

## Instalar paquetes
Podemos utilizar funciones mágicas de los Jupyter Notebooks (ipython) para
instalar los paquetes necesarios, ya que asi nos aseguramos que estos sean
instalados en el ambiente correspondiente.

In [ ]:
%pip install --upgrade pandas-gbq 'google-cloud-bigquery[bqstorage,pandas]'
%pip install ipywidgets

In [ ]:
%pip install bigframes

In [ ]:
%pip install --upgrade pandas

In [2]:
!which python3

/home/inspired/data-science-on-gcp/dsongcp/bin/python3


## Importar librerías

In [3]:
"""
Desarrollo en notebook local.

Desarrollamos en un notebook local para analizar el procedimiento a seguir
para guardar la tabla en un archivo
"""

from google.cloud import bigquery
import pandas as pd
import pandas_gbq
import bigframes.pandas as bpd

## Consultar en Bigquery

Para hacer consultas en BiQuery y guardarlas en un dataframe en python podemos
emplear 3 métodos que prácticamente son lo mismo:

1. Llamadas a BigQuery utilizando la API de python:
   Primero creamos el objeto cliente de BigQuery usando su constructor
   Client() luego usamos métodos de ese cliente para lo necesario, en
   este caso utilizamos el método query_and_wait y luego al iterador
   devuelto le aplicamos el método to_dataframe.

2. Módulo pandas_gbq:
   El cual proporciona una envoltura para el servicio web de análisis
   BigQuery de Google, en este caso utilizamos la función read_gbq del módulo.

3. Line magics o Cell magics
   Acá las lineas o celdas de magia son funciones de ipython que se pueden
   llamar con un estilo similar al sintaxis de la linea de comandos, si la
   función no es predefinida debemos cargarla si estamos en nuestro entorno
   local utilizando el nombre de la extensión.


### Alternativas de descarga:

Se definen las variables a utilizar en las alternativas como el string de
la consulta y otras según sea necesario para el tipo de consulta.

En este caso solamente definimos el string de la consulta.

In [4]:
SQL = """
SELECT
    *
FROM bigquery-manu-407202.dsongcp.flights
"""

#### Alternativa 1 - Llamadas a BigQuery

Creamos el objeto cliente y ejecutamos la consulta.

In [5]:
client = bigquery.Client()
# query retorna QueryJob
q_result = client.query(SQL)

Creamos el DataFrame

In [6]:
df = q_result.to_dataframe(date_dtype=pd.StringDtype())

[Alternativa Opcional]
query_and_wait retorna RowIterator

In [ ]:
q_result_wait = client.query_and_wait(SQL)
df = q_result_wait.to_dataframe(date_dtype=pd.StringDtype())

In [ ]:
df = client.query_and_wait(SQL).to_dataframe(date_dtype=pd.StringDtype())

Creamos los DataFrames de manera separada para evitar sobrecargar la
memoria local

In [ ]:
# problemas con formato de fecha queda como datetime
df_bq = q_result.to_dataframe()

In [ ]:
# problemas con formato de fecha queda como date
df_bq_date = q_result.to_dataframe(date_dtype=None)

In [ ]:
# Correcto como str
df_bq_string_date = q_result.to_dataframe(date_dtype=pd.StringDtype())

#### Alternativa 2 - Pandas Google Big Query

Utilizamos read_gbq para leer la consulta directamente en un DataFrame con
la opción de use_bqstorage_api en True para utilizar la API de
BigQuery Storage y así obtener resultados grandes en menor tiempo.

In [ ]:
# pyright: reportAssignmentType = false
df_bq_pandas: pd.DataFrame = pandas_gbq.read_gbq(SQL, use_bqstorage_api=True)

#### Alternativa 3 - Jupyter Magic

El sintaxis de la celda mágica es el siguiente:

```
%%bigquery [<destination_var>] [--project <project>] [--use_legacy_sql]
           [--verbose] [--params <params>] <query>
```

Obs. si nos encontramos en un ambiente local, antes de ejecutar la celda
debemos ejecutar la linea mágica load_ext la cual carga las funciones
mágicas por su nombre de módulo.

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
%%bigquery df_bq_magic
SELECT
    *
FROM bigquery-manu-407202.dsongcp.flights

Descargamos el esquema de la tabla flights y borramos los campos extras
que describen la tabla, utilizamos show para mostrar información de un
recurso.

In [ ]:
!bq show --format=prettyjson dsongcp.flights > flights_schema.json

### Leer archivos descargados desde la consola

Leemos los archivos descargados desde la consola de google cloud platform
para comparar los formatos y que debemos cambiar.

Obs. dependiendo del motor para leer los archivos ya sea `ujson` o `pyarrow`
los tipos de datos serán distintos, esto también depende del back-end de
tipos de datos `numpy_nullable` retorna un DataFrame y `pyarrow` retorna
ArrowDtype.

Prueba archivo json pequeño `flights_few_lines.json`

In [4]:
!head -10 flights/flights_console.json > flights/flights_few_lines.json

In [ ]:
# pyright: reportCallIssue = false
df_few_json = pd.read_json("flights/flights_few_lines.json",
                               orient="records",
                               lines=True,
                               engine="pyarrow"
                               )
df_few_json.info()

Archivo json `flights_console.json`

In [ ]:
# pyright: reportCallIssue = false
df_console_json = pd.read_json("flights/flights_console.json",
                               orient="records",
                               lines=True,
                               engine="pyarrow"
                               )
df_console_json.info()

In [ ]:
df_console_json_10=df_console_json.head(10)
df_console_json_10

Prueba archivo csv pequeño `flights_few_lines.csv`

In [5]:
!head -10 flights/flights_console.csv > flights/flights_few_lines.csv

In [ ]:
df_console_csv_fl = pd.read_csv("flights/flights_few_lines.csv")
df_console_csv_fl.info()

Archivo `flights_console.csv`

In [ ]:
df_console_csv = pd.read_csv("flights/flights_console.csv")
df_console_csv.info()

### Escribir a un archivo local

Escribimos a un archivo local después de analizar.

Analizamos probando si escribiendo a los archivos con pocos datos nos
otorga buenos resultados

In [6]:
df_10 = df.head(10)
json = df_10.to_json(
    "flights/flights_10.jsonl",
    orient="records",
    lines=True,
)

Escribimos a un archivo local después de analizar.

In [ ]:
json = df.to_json(
    "flights/flights.json",
    orient="records",
    lines=True,
)

In [8]:
chunk_size = 300000  # Define el tamaño de cada trozo
for i in range(0, len(df), chunk_size):
    chunk = df[i:i + chunk_size]
    chunk.to_json(f"flights/chunks/flights_{i}.jsonl", orient="records", lines=True)

In [7]:
chunk_size = 300000  # Define el tamaño de cada trozo
num_chunks = len(df) // chunk_size + 1  # Calcula el número total de trozos

for i in range(0, len(df), chunk_size):
    chunk = df[i:i + chunk_size]
    chunk.to_json(
        f"flights/chunks/flights_{str(i // chunk_size).zfill(5)}-of-{str(num_chunks).zfill(5)}.jsonl",
        orient="records",
        lines=True,
    )

Test

In [ ]:
df_bq_10 = df_bq.head(10)
json = df_bq_10.to_json(
    "flights/df_bq_10.json",
    orient="records",
    lines=True,
)

In [ ]:
df_bq_date_10 = df_bq_date.head(10)
json = df_bq_date_10.to_json(
    "flights/df_bq_date_10.json",
    orient="records",
    lines=True,
)

In [ ]:
df_bq_string_date_10 = df_bq_string_date.head(10)
json = df_bq_string_date_10.to_json(
    "flights/df_bq_string_date_10.json",
    orient="records",
    lines=True,
)

### [Alternativa Opcional] Modificaciones
Hacemos los cambios correspondientes, si es que no especificamos al momento de
transformar a DataFrame

In [ ]:

df_bq["FL_DATE"] = pd.to_datetime(df_bq["FL_DATE"])
df_bq["FL_DATE"] = df_bq["FL_DATE"].dt.strftime('%Y-%m-%d')

# Test
not_string_cols = ['FL_DATE', 'DEP_DELAY', 'TAXI_OUT',
                   'TAXI_IN', 'ARR_DELAY', 'CANCELLED', 'DIVERTED']
string_cols = [col for col in df_bq.columns if col not in not_string_cols]
for col in string_cols:
    df_bq[col] = df_bq[col].astype(str)

# Completar con ceros a la izquierda las columnas con formato "hhmm"
for col in ["CRS_DEP_TIME", "DEP_TIME", "WHEELS_OFF", "WHEELS_ON", "CRS_ARR_TIME", "ARR_TIME"]:
    df_bq[col] = df_bq[col].str.zfill(4)

# Eliminamos los vuelos cancelados y desviados con fines de desarrollo
df_bq = df_bq.loc[~df_bq["DIVERTED"] & ~df_bq["CANCELLED"]]

### Desarrollo airports
para el archivo `airports_2024`

In [ ]:
df_csv = pd.read_csv("airports_2024.csv")
airports_tz = df_csv.iloc[:, [0, 21, 26]]
airports_tz